In [17]:
from datasets import load_dataset, Dataset

data = load_dataset("csv",data_files="daigt_external_dataset.csv")["train"]

print(data)

Dataset({
    features: ['id', 'text', 'instructions', 'source_text'],
    num_rows: 2421
})


In [32]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("distilbert/distilroberta-base")
model = AutoModelForSequenceClassification.from_pretrained(
    "distilroberta-base", num_labels=2
)

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [33]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
           

In [ ]:
import numpy as np

data_human = data.select_columns(["text"])
data_human = data_human.add_column("label", np.zeros(len(data_human), dtype=int))

data_ai = data.select_columns(["source_text"])
data_ai = data_ai.add_column("label", np.ones(len(data_ai), dtype=int))

data_human = data_human.rename_column("text", "content")
data_ai = data_ai.rename_column("source_text", "content")

final_dataset = Dataset.from_dict(
    {
        "content": list(data_human["content"]) + list(data_ai["content"]),
        "label": list(data_human["label"]) + list(data_ai["label"]),
    }
)

In [34]:
def preprocess_text(example):
    return tokenizer(
        example["content"],
        truncation=True,
        padding="max_length",
        max_length=256,
    )
tokenized_dataset = final_dataset.map(preprocess_text,batched=True)

test = tokenized_dataset.train_test_split(test_size=0.2,seed=42)
train_val = test["train"].train_test_split(test_size=0.1,seed=42)

train_dataset = train_val["train"]
test_dataset = test["test"]
val_dataset = test["test"]

Map:   0%|          | 0/4842 [00:00<?, ? examples/s]

In [ ]:
# pred metrics
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average="binary"
    )
    accuracy = accuracy_score(labels, preds)

    return {"accuracy": accuracy, "f1": f1, "precision": precision, "recall": recall}

In [38]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments(
    output_dir="./",
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    greater_is_better=True,
    logging_dir="./logs",
    logging_steps=50,
    report_to="wandb",
    run_name="ai_text_detector"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

trainer.train()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/tmp/ipykernel_53030/2301943798.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


wandb: Currently logged in as: dattucodes (death-star) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.021100,0.021969,0.993808,0.993915,0.987903,1.000000
2,0.009100,0.015205,0.995872,0.995935,0.991903,1.000000
3,0.000100,0.007680,0.997936,0.997963,0.995935,1.000000


TrainOutput(global_step=654, training_loss=0.0321448334315717, metrics={'train_runtime': 122.4701, 'train_samples_per_second': 85.368, 'train_steps_per_second': 5.34, 'total_flos': 692473326474240.0, 'train_loss': 0.0321448334315717, 'epoch': 3.0})